<a href="https://colab.research.google.com/github/JMindpalace/Machine_Learning/blob/main/3.0%20%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

< 해당 페이지는 **특성(Feature)**을 파악하고, **특성 공학(Feature Engineering)**으로 증폭 혹은 축소 시키는 페이지입니다. >

> 데이터 변경은 거의 없고, 모델링의 전처리로 특성공학이나 상관계수 등에 따라 줄이는 것이 목표입니다.

> 기대 결과 값: 여러 특성 간의 상관관계를 말할 수 있습니다.

# 데이터 분석(Data Analysis)

## 데이터 분석의 방향성

- 지도학습적 목적 : 특성과 타겟의 관계를 파악

- 비지도학습적 목적 : 특성간의 관계를 파악해서 예측 타겟 설정



---



---



### 데이터셋 분석

In [ ]:
# 특정 조건 검색1
df[ (df['col'] == 'condition') ] # []내에 ()와 &로 다중 조건 검색 가능

# 특정 조건 검색2 - 쿼리문의 변수는 @로 가져와야함(@변수)
df.query( ('col' == 'condition') ) # ()내에 ()와 and로 다중 조건 검색 가능

# Groupby - mean()외에도 median() 등 사용 가능
df.groupby('그룹 기준 칼럼').mean()['값을 볼 칼럼'] # [기준1, 기준2]로 대/중분류, [ [값1, 값2] ]로 다중 그룹
# Groupby가 리스트 반환이지만, 기준 칼럼 뒤에 , as_index=False면 데이터 프레임으로 반환



---



---



# 과대적합 감소

## 하이퍼-파라미터 조정



---



---



## 특성 조정(Feature Importance)
> 모델의 예측값(성능)에 대한 특성들의 중요도 확인<br>
> 낮은 중요도는 Drop

### MDI(Mean Decrease impurity)
> 트리 기반 모델에서 사용

In [ ]:
pipe = make_pipeline( OrdinalEncoder(),RandomForestClassifier() )
rf = pipe.named_steps['randomforestclassifier'] # rf로 파이프라인 속성에 접근
importances = pd.Series(rf.feature_importances_, X_train.columns) # 특성 이름 , 특성 중요도
importances.sort_values().plot.barh() # MDI 시각화

In [ ]:
# 단, MDI의 추가적인 단점은 트리기반이기에 Cardinality의 의존도가 높다는 점이다
X_train.nunique().sort_values().plot.barh() # Cardinality 그래프화

### Drop-Column Importance
> 모든 특성을 가진 모델(기준) - 특정 특성을 제거하고 재학습한 모델<br>
> 평가 하락시 중요 특성으로 파악함

In [ ]:
base_score = pipe.score(X_test, y_test)

dci = pd.Series(dtype=float)
for i in features: # features는 target과 분리 시 사용
  p = pipe()

  p.fit(X_train.drop(columns=[i], axis=1), y_train) # 특성 1개 drop 후 모델 재학습 - 큰 단점
  score_dropped = p.score(X_test.drop(columns=[i], axis=1), y_test) # 특성 1개 drop한 모델의 점수
  dci[feature] = score - score_dropped

dci.sort_values().plot.barh()

### Permutation Importance(순열 중요도)
> 기준모델에서 특성마다 노이즈를 주어 성능 감소폭 확인<br>
> 모든 모델에서 적용이 가능함

In [ ]:
pi, n_iter = pd.Series((dtype=float), 10 # 성능 확인 과정 반복 횟수

for i in features:  # 노이즈를 줄 feature 선택
  X_test_permed = X_test_copy() # 중첩에서 변수값 변경
  scores_permutated = []

  for _ in range(n_iter):
    X_test_permed[i] = np.random.permutation(X_test_permed[i])
    scores_permutated.append( pipe.score(X_test_permed, y_test) )
  avg_score = np.mean(scores_permutated) # 성능들의 평균 점수
  pi[features] = score - avg_score       # 성능 하락폭
pi.sort_values().plot.barh()

# eli5 라이브러리 사용
permuter = PermutationImportance(
    pipe.named_steps["randomforestclassifier"],  # model
    scoring="accuracy", n_iter=10, random_state=2
)
X_test_eli5 = pipe[0].transform(X_test) # OrdinalEncoder 사용
permuter.fit(X_test_eli5, y_test)

feature_names = X_test.columns.tolist()
pd.Series( permuter.feature_importances_, feature_names ).sort_values().plot.barh()

eli5.show_weights( # 특성별 score
    permuter,
    top=None,  # top n 지정 가능, None 일 경우 모든 특성
    feature_names=feature_names,  # list 형식
)